In [ ]:
import xarray as xr

ds_GFD_info = xr.open_dataset(
    "/home/grupos/geocean/faugeree/BlueMath/methods/hybrid_downscaling/additive/GreenSurge/outputs/GreenSurge_Tonga_info_updated.nc",
    decode_times=False,
)
ds_GFD_info

<div style="border: 1px solid #cce5ff; background-color: #e9f7fe; padding: 15px; border-radius: 6px; font-family: sans-serif;">
  <strong>🔹 Model validation</strong><br><br>

  The figures below show the maps of a given time and the swath maps of wind setup resulting from dynamic simulations with the Shallow Water Equation (SWE) model Delft3D (left panels) and from the GreenSurge aproach (right panels). These figures illustrate the importance of the wind setup in shallow water areas close to shore (it reaches values up to 0.2m whithin this case) and de acuracy of the GreenSurge approach compared to dynamic simulations.
</div>

In [ ]:
xds_GS_wind =xr.open_dataset("/lustre/geocean/DATA/extra_work/faugeree/Tonga_GreenSurge/GreenSurge_Postprocess/GreenSurge_wind_HAROLD2020.nc")

ds_WL_dynamic_WindSetUp = xr.open_dataset(
    "/home/grupos/geocean/faugeree/BlueMath/methods/hybrid_downscaling/additive/GreenSurge/inputs/GreenSurge_GFDcase_map.nc"
)

In [ ]:
from bluemath_tk.additive.greensurge import plot_GS_vs_dynamic_windsetup_swath

plot_GS_vs_dynamic_windsetup_swath(
    xds_GS_wind, ds_WL_dynamic_WindSetUp, ds_GFD_info, vmin=0, vmax=0.3
)

In [ ]:
from bluemath_tk.additive.greensurge import plot_GS_vs_dynamic_windsetup

plot_GS_vs_dynamic_windsetup(
    xds_GS_wind,
    ds_WL_dynamic_WindSetUp,
    ds_GFD_info,
    ds_WL_dynamic_WindSetUp.time[160].values,
)

<div style="border: 1px solid #cce5ff; background-color: #e9f7fe; padding: 15px; border-radius: 6px; font-family: sans-serif;">
  <strong>🔹 Inverse Barometer</strong><br><br>

  The third stage for any TC event study consist of turning pressure differences into sea level rises and falls using IB methodology which take into account the correlation 1mb-1cm.
</div>


In [ ]:
from bluemath_tk.additive.greensurge import pressure_to_IB

ds_WL_GS_IB = pressure_to_IB(xds_vortex_GS)

In [ ]:
from bluemath_tk.additive.greensurge import plot_GS_TG_validation_timeseries

tide_gauge = xr.open_dataset(
    "/home/grupos/geocean/faugeree/BlueMath/methods/hybrid_downscaling/additive/GreenSurge/inputs/TG_HAROLD2020_Tongatapu.nc"
)

plot_GS_TG_validation_timeseries(
    xds_GS_wind,
    ds_WL_GS_IB,
    ds_WL_dynamic_WindSetUp,
    tide_gauge,
    ds_GFD_info,
)